# Lista 2 - Raizes de função


## Importando os pacotes

In [1]:
import numpy as np
from methods.bracketing_methods import bissection, false_position
from methods.open_methods import newton_raphson, secant, fixed_point
import pandas as pd
import sympy as sy

## Configuração

In [ ]:
#func: str = "x - tan(x)"
#intervals = [[4.4, 4.7], [7, 7.75], [10, 10.99], [13, 14.13], [16, 17.27]]
tol: np.float64 = np.float64(0.0001)
max_iter: int = 100

func:str = "(x/2) - tan(2*x)"
intervals = [[1.5, 2.1], [3.0, 3.7], [5.0, 5.35], [6.4, 7.0], [9.5, 10.2]]

csv_id = 1 if func == "x - tan(x)" else 2

## Metodos Intervalares


In [3]:
bracketing_methods = [bissection, false_position]
bracketing_methods_names = ["Bissection", "False Position"]

result_df = pd.DataFrame()

for method, method_name in zip(bracketing_methods, bracketing_methods_names):
    result = {
        'root_index': [],
        "xl": [],
        "xu": [],
        "x": [],
        "relative_error": [],
        "tol": [],
        "iter": [],
        "function": [],
    }

    root_index = 0

    for interval in intervals:
        xl = np.float64(interval[0])
        xu = np.float64(interval[1])

        for i in range(1, max_iter + 1):
            x, xl_returned, xu_returned, relative_error, iter,  = method(
                func, xl, xu, tol, i
            )

            result["root_index"].append(root_index)
            result["xl"].append(xl_returned)
            result["xu"].append(xu_returned)
            result["x"].append(x)
            result["relative_error"].append(relative_error)
            result["tol"].append(tol)
            result["iter"].append(iter)
            result["function"].append(func)

            if relative_error < tol:
                break

        root_index += 1

    tmp_df = pd.DataFrame(result)
    tmp_df["method"] = method_name

    result_df = pd.concat([result_df, tmp_df], ignore_index=True)

## Metodos Abertos


### Ponto Fixo


In [4]:
g_from_function: str = "2 * tan(2 * x)" # Não converge

result = {
    "root_index": [],
    "xl": [],
    "xu": [],
    "x": [],
    "relative_error": [],
    "tol": [],
    "iter": [],
    "function": [],
}

root_index = 0

for interval in intervals:
    xl = np.float64(interval[0])

    for i in range(1, max_iter + 1):
        x, x_old, relative_error, iter = fixed_point(g_from_function, xl, tol, i)


        result["root_index"].append(root_index)
        result["xl"].append(x_old)
        result["xu"].append(x_old)
        result["x"].append(x)
        result["relative_error"].append(relative_error)
        result["tol"].append(tol)
        result["iter"].append(iter)
        result["function"].append(func)

        if relative_error < tol:
            break

    root_index += 1

tmp_df = pd.DataFrame(result)
tmp_df["method"] = "Fixed Point"

result_df = pd.concat([result_df, tmp_df], ignore_index=True)

### Newton Raphson


In [5]:
derivative = sy.diff(func, sy.symbols("x"))  # Calcula a derivada
derivative = sy.lambdify(sy.symbols("x"), derivative, "numpy")  # Converte para função

# Ponto de inflexão na vizinhança

result = {
        "root_index": [],
        "xl": [],
        "xu": [],
        "x": [],
        "relative_error": [],
        "tol": [],
        "iter": [],
        "function": [],
    }

root_index = 0

for interval in intervals:
    xl = np.float64(interval[1])

    for i in range(1, max_iter + 1):
        x, x_old, relative_error, iter = newton_raphson(func, derivative, xl, tol, i)

        result["root_index"].append(root_index)
        result["xl"].append(x_old)
        result["xu"].append(x_old)
        result["x"].append(x)
        result["relative_error"].append(relative_error)
        result["tol"].append(tol)
        result["iter"].append(iter)
        result["function"].append(func)

        if relative_error < tol:
            break

    root_index += 1

tmp_df = pd.DataFrame(result)
tmp_df["method"] = "Newton Raphson"

result_df = pd.concat([result_df, tmp_df], ignore_index=True)

### Secante

In [6]:
result = {
        "root_index": [],
        "xl": [],
        "xu": [],
        "x": [],
        "relative_error": [],
        "tol": [],
        "iter": [],
        "function": [],
    }

root_index = 0

for interval in intervals:
    xl = np.float64(interval[0])
    xu = np.float64(interval[1])

    for i in range(1, max_iter + 1):
        x, xl_returned, xu_returned, relative_error, iter = secant(func, xl, xu, tol, i)

        result["root_index"].append(root_index)
        result["xl"].append(xl_returned)
        result["xu"].append(xu_returned)
        result["x"].append(x)
        result["relative_error"].append(relative_error)
        result["tol"].append(tol)
        result["iter"].append(iter)
        result["function"].append(func)

        if relative_error < tol:
            break

    root_index += 1

tmp_df = pd.DataFrame(result)
tmp_df["method"] = "Secant"

result_df = pd.concat([result_df, tmp_df], ignore_index=True)

In [7]:
result_df.to_csv(f"../output/methods_result_{csv_id}.csv", sep=",", index=False, float_format="%.10f")
#result_df.to_parquet("../output/methods_result_1.parquet", index=False)